# How much of the country has sub-broadband Internet access?


In [1]:
import pandas as pd

In [2]:
# Load data 
data = pd.read_csv("data/fbd_us_with_satellite_jun2015_v3.csv")

In [3]:
data.head(3)

,LogRecNo,Provider_Id,FRN,ProviderName,DBAName,HoldingCompanyName,HocoNum,HocoFinal,StateAbbr,BlockCode,TechCode,Consumer,MaxAdDown,MaxAdUp,Business,MaxCIRDown,MaxCIRUp
0,1,10899,8702680,ADIR International Export Ltd d/b/a La Curacao,Curatel,"Curatel, LLC",190014,"Curatel, LLC",CA,6.037102e+13,10,1,0.8,0.4,0,0,0
1,2,10899,8702680,ADIR International Export Ltd d/b/a La Curacao,Curatel,"Curatel, LLC",190014,"Curatel, LLC",CA,6.037102e+13,10,1,1.5,0.8,0,0,0
2,3,10899,8702680,ADIR International Export Ltd d/b/a La Curacao,Curatel,"Curatel, LLC",190014,"Curatel, LLC",CA,6.037104e+13,10,1,4.0,1.5,0,0,0


In [4]:
# Return only the records where Consumer is set to 1 and MaxAdDown > 0
def filter_data(df):
    return df[(df["Consumer"] == 1) & (df["MaxAdDown"] > 0)]
 
print "Started with " + str(len(data.index)) + " records."
print "Narrowed down to " + str(len(filter_data(data).index)) + " records."


Started with 38231692 records.
Narrowed down to 31045011 records.


In [5]:
def by_block(df):
    return df.groupby("BlockCode").agg({"MaxAdDown":max,"MaxAdUp":max})

def get_by_block(df):
    df = by_block(filter_data(df))
    return df

print "Found " + str(len(get_by_block(data))) + " block records."

Found 9512417 block records.


In [7]:
get_by_block(data)["MaxAdUp"].value_counts().sort_index()

0.064            1
0.102            2
0.120            3
0.125            1
0.128         5421
0.152            1
0.155            1
0.159            2
0.160            2
0.190          905
0.200        74725
0.220            1
0.250          799
0.256        20465
0.280            4
0.300           77
0.310          111
0.352            5
0.356            3
0.368            1
0.375          107
0.380         4172
0.384        37979
0.386            1
0.400        19652
0.440          292
0.459            1
0.500         6932
0.507            2
0.509            1
             ...  
49.000        5557
50.000      218441
51.000        5137
52.000           6
60.000         318
70.000         397
75.000        2403
80.000           8
90.000         146
99.000           1
100.000     574550
102.000          1
105.000         15
120.000       4616
125.000       1651
135.000          2
150.000        598
155.000      13562
157.000          2
160.000          1
200.000       1124
210.000     

In [ ]:
get_by_block(data).apply(lambda x: round(x["MaxAdUp"]), axis=1).value_counts().sort_index()

In [30]:
import json
def get_by_state(df):
    ret = {}
    for st in df["StateAbbr"].unique():
        state_df = get_by_block(df[df["StateAbbr"] == st])
        total_blocks = len(state_df.index)
        slow_blocks = len(state_df[(state_df["MaxAdDown"] < 25) | (state_df["MaxAdUp"] < 3)].index)
        if total_blocks > 0:
            pct_slow = slow_blocks * 100 / total_blocks
        else:
            pct_slow = 0
        ret[st] = {
            "slow": slow_blocks,
            "total": total_blocks,
            "pct_slow": pct_slow
        }
    return ret
state_summary = pd.DataFrame(get_by_state(data)).transpose().reset_index()


In [45]:

state_summary_sorted = state_summary.sort_values(by="pct_slow").sort_values(by="pct_slow")
state_summary_sorted.columns = "state","pct slow","slow blocks","total blocks"

#state_summary.columns = "state","pct slow","slow blocks","total blocks"
#state_summary.reset_index()[["index","pct_slow"]].to_csv(sep="\t",index=False)

state_summary_txt = state_summary_sorted[["state","pct slow"]]
state_summary_txt["slow blocks"] = state_summary_txt.apply(lambda x: str(x["pct slow"]) + "%", axis=1)
print state_summary_txt[["state","slow blocks"]].to_csv(sep="\t",index=False)

state	slow blocks
	0%
NJ	2%
RI	2%
MA	7%
WA	8%
NY	8%
VI	10%
NC	12%
NH	15%
MD	15%
CT	16%
PA	17%
TN	21%
OH	21%
OR	22%
DC	23%
SC	23%
SD	24%
ME	25%
ND	29%
UT	29%
MI	30%
DE	30%
VA	34%
HI	35%
NV	35%
TX	36%
WI	36%
LA	37%
PR	37%
MN	38%
CA	40%
FL	41%
IL	42%
VT	42%
GA	44%
MS	46%
WV	47%
KY	48%
MO	48%
IN	48%
AL	50%
CO	52%
KS	53%
MT	59%
NE	59%
IA	60%
AR	62%
AZ	63%
NM	63%
OK	65%
WY	71%
ID	73%
AK	76%
GU	99%
AS	100%
MP	100%



/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
ct_data = pd.read_csv("data/CT-Fixed-Jun2015-v3.csv")
pd.DataFrame(get_by_state(ct_data)).transpose()

,pct_slow,slow,total
CT,16,10951,65560


In [14]:
len(data[data["StateAbbr"] == "CT"].index)

427792

In [15]:
len(ct_data.index)

427792

In [46]:
state_summary.sum()

pct_slow       2228
slow        3651832
total       9512417
dtype: int64

In [48]:
state_summary["slow"].sum() * 100 / state_summary["total"].sum()

38